* This takes data from the SQL database, and saves it in a condensed trajectory form
* This file is included for completeness, but it is not needed to reproduce the results in the paper, and also it will not run (needs database access)

In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle
import glob
import gzip
import time
import pandas as pd
import scipy.stats
import sys
import multiprocessing
import psycopg2

import os

import warnings
warnings.filterwarnings('ignore')

In [18]:
year = 2019
%cd '/sharedcodes/bees/code/'

if year==2018:
    import definitions_2018 as bd
    resultsdir = '/data/beeresults/'
    comb_contents_dir = '/data/comb-contents-images/'
elif year==2019:
    import definitions_2019 as bd
    resultsdir = '/data/beeresults2019/'
    comb_contents_dir = '/data/comb-contents-images2019/'
    
import displayfunctions as bp  # 'bee plots'
import datafunctions as dfunc
dfunc.init(bd) 
bp.init(bd)
bd.year

/sharedcodes/bees/code


2019

In [19]:
### One day is too long to grab at once.  Better to do one hour an iterate through. 

In [20]:
# days lived - import this, because will use to only save data for bees that were alive on a given day
# This is processed in '2- Create Data Matrix', separately for each year
[cohort_daysalive] = pickle.load(open(resultsdir+'daysalive.pkl','rb'))
uid_daysalive = np.concatenate(cohort_daysalive)

In [21]:
def dbquery(bee_ids,day_input,bee_id_confidence_threshold=0.3,starttime = "00:00:00.000000+00:00",endtime = "23:59:59.999999+00:00",limit=44236800):  # this limit is all bees, tracked for one hour    
    # bee_ids:  list of tag ids
    day =  pd.Timestamp(day_input,freq='D')
    if len(bee_ids)==0:  # none are in the list, so select none
        bee_id_string = " bee_id IN (-1) AND "
    elif len(bee_ids)==4096:
        bee_id_string = ''
    elif len(bee_ids)==1:
        bee_id_string = " bee_id = "+str(bee_ids[0])+" AND "
    else:
        bee_id_string = " bee_id IN "+str(tuple(bee_ids))+" AND "
    
    if starttime<endtime:
        daystringstart = day.strftime('%Y-%m-%d')
        daystringend = daystringstart
    else:
        daystringstart = day.strftime('%Y-%m-%d')
        day2 = day + day.freq  #  this is what the 'proper' way to do it is, because freq = 1 Day
        daystringend = day2.strftime('%Y-%m-%d')


    conn = psycopg2.connect(bd.querydata)
    df = pd.read_sql("SELECT * FROM "+bd.databasename+" WHERE " 
                 +bee_id_string
                     +"bee_id_confidence>"+str(bee_id_confidence_threshold) 
                 +" AND timestamp BETWEEN '" +daystringstart+" "+starttime+ "' AND '"+daystringend+" "+endtime+ "' "
                 +"ORDER BY timestamp " + 
                "LIMIT "+str(limit), 
                 conn, coerce_float=False) 
    return df  

# (1) Save condensed form of trajectory data

In [22]:
numtimedivs=24 # how much to get from the database at once
saveskip=1  # e.g 6 = every 2 sec

if numtimedivs==24:
    prefix = 'dayhour'
elif numtimedivs==288:
    prefix = 'day5min'
else:
    prefix = 'day1min'

# I made these stricter after the first time I ran with the data.  I didn't systematically test/analyze though
conf_threshold = 0.8

min_num_obs_hour=10  # min number of obs in an hour to calculate quantities for that hour.  If don't have any data points, then the calculations don't make sense.

tagids = np.arange(bd.numbees)  # this is tag id = tid, NOT unique id (uid)

framesperhour = int(3*60*60)


def processday_dftraj(daynum):
    day = bd.alldaytimestamps[daynum]
    print('day: ',day)    
    
#     comb = dfunc.day_comb_data(comb_contents_dir,daynum)
    comb = pickle.load(gzip.open(comb_contents_dir+'comb_'+str(daynum).zfill(3 if year==2019 else 2)+'.pklz','rb'))

    # Initialize an array to save movement data for bees for this day
    # median speed, median abs value angular velocity, time active
    # these will keep a values of -1 if there isn't enough data to do calculations

    dfblank = pd.DataFrame([],columns=['daynum','framenum','uid','x','y','camera'])
    
    # get the bees that were alive this day.  Note, need to convert to uids, instead of tagids
    age = np.concatenate(dfunc.getages(daynum))  # age per uid
    uids_this_day = dfunc.convert_tagids_to_uids(np.arange(bd.numbees),daynum)  # this returns the first one, sorted by age

    # don't use dayobservesel for this - therefore can put above instead of below loop
    alive_sel = (age>=0) & (age<=uid_daysalive)  #& dayobservedsel_byuid
    uids_alive = np.where(alive_sel)[0]
    # make a mask, because its a more efficient way of selecting from huge arrays
    tags_alive = dfunc.sel_cohort_bee(np.arange(bd.numbees))[alive_sel]  # note: only do this for a single day
    tags_alive_mask = np.tile(False,bd.numbees)
    tags_alive_mask[tags_alive] = True        
    
    # loop over time divisions
    dftemps = [dfblank]
    for timenum in range(numtimedivs):     
        print(timenum)
        minperdiv = np.floor(24*60 / numtimedivs).astype(int)
        hour0, minute0 = np.divmod(minperdiv*timenum,60)
        hour1, minute1 = np.divmod(minperdiv*(timenum+1),60)
        starttime = str(hour0).zfill(2)+':'+str(minute0).zfill(2)+':00.000000'
        endtime = str(hour1).zfill(2)+':'+str(minute1).zfill(2)+':00.000000'             
        
        # query all
        df = dbquery(tagids,day,starttime=starttime,endtime=endtime,limit=12052340*4)
        # always use df_to_coords, because it filters duplicates, and also applys conversion factor to x and y for 2019
        # 21 Jan 2022 Update: df_to_coords now calculates integer framenum, and filters by this. This corrects camera-switching false id errors
        all_camera,all_x,all_y,all_theta,all_ids,all_times,all_framenums,all_conf = dfunc.df_to_coords(df,conf_threshold=conf_threshold)        
        # Note:  could change the code below to use all_frames to count interactions.  BUT, they are not sorted by time, because of the different cameras.  So, would have to change the whole selection process, and it doesn't really matter, so just keep it
        
        # loop over bees:  Get quantities per-hour
        for beenum, bee_uid in zip(tags_alive,uids_alive):
            idsel = (all_ids == beenum)
            numobs = np.sum(idsel)
            if numobs>=min_num_obs_hour:
                camera,x,y,theta,ids,times,framenums = all_camera[idsel],all_x[idsel],all_y[idsel],all_theta[idsel],all_ids[idsel],all_times[idsel],all_framenums[idsel]

                ### Calculate speed and filter by max speed.  This removes "jumps", or huge speed values, that are definitely tracking errors
                # Michael said max speed from manual tracking were:  Max speed: 0.905±0.510 cm s−1 in one set of colonies, 1.071±0.438 cm s−1 in another set of colonies.  So, this value is conservative (high), to err on the side of keeping more data points, but probably does not matter
                # Using max speed of 3 cm/sec, which is higher than any ever should have
                max_speed = 3 * bd.pixels_per_bin   
                # need to do an iterative procedure to remove error frames regarding speed, because the difference between points could still be very large after a single point is removed
                numerrorframes = 1
                while (numobs>=min_num_obs_hour) & (numerrorframes>0):
                    dtimes_temp = np.diff(times).astype(float) * 10**-9
                    speed_temp = dfunc.getflatdistance(x,y,camera) / dtimes_temp
                    samecamera_temp = (np.diff(camera) == 0)
                    error_frames = (speed_temp>max_speed) & samecamera_temp
                    tokeep_sel = np.logical_not(np.concatenate(([True if error_frames[0]==True else False],error_frames)))
                    numerrorframes = np.sum(np.logical_not(tokeep_sel))  # do this instead of summing error_frames, to keep the contingency for the 1st frame
                    numobs = np.sum(tokeep_sel)
                    camera,x,y,theta,ids,times,framenums = camera[tokeep_sel],x[tokeep_sel],y[tokeep_sel],theta[tokeep_sel],ids[tokeep_sel],times[tokeep_sel],framenums[tokeep_sel]
                # account for the 2019 "time shift" of database queries.  The 'time' in the df is UTC time, but queries are assumed in UTC+2.
                # So, if query 1am, it will return 23:00 the previous day.
                # Account for this by shifting the framenumbers up two hours, and then using mod for wrap around
                # With this, then the query returning 23:00 the previous day, has a framenumber for 01:00 the current day, which is correct
                # and 22:00 the current day gets shifted to 24:00
                if year==2019:
                    framenums = np.mod(framenums + framesperhour*2,framesperhour*24)
                # use this to select framenumbers if saving skipped trajectory versions
                framenumsel = np.tile(False,len(framenums))
                framenumsel[np.mod(framenums,saveskip)==0] = True
                dftemp = dfblank.copy()
                dftemp['framenum'] = framenums[framenumsel]
                dftemp['x'] = x[framenumsel]
                dftemp['y'] = y[framenumsel]
                dftemp['theta'] = theta[framenumsel]
                dftemp['camera'] = camera[framenumsel]
                dftemp['uid'] = bee_uid
                dftemp['daynum'] = daynum                    
                dftemps.append(dftemp)
    
    dfcat = pd.concat(dftemps)
    dfcat.reset_index(inplace=True)
    dfcat.drop(columns=['index'],inplace=True)


    print('writing to file....')
    # output the results for this day as a pkl file
    # savefile = resultsdir+'beetrajectories_skip'+str(saveskip)+'_'+str(daynum).zfill(3)+'.pklz'
    # pickle.dump(dfcat, gzip.open(savefile,'wb'),protocol=4)
    
    ### updated:  output directly as an HDF file
    columns = ['daynum', 'framenum', 'uid', 'x', 'y', 'camera']
    dfcat.loc[:,columns] = df[columns].applymap(int)
    outfile = resultsdir+'beetrajectories_'+str(daynum).zfill(3)+'.hdf'
    dfcat.to_hdf(outfile,complevel=9,key='df', mode='w')  

    print('wrote to:', savefile)


In [23]:
daystoprocess= np.arange(0,bd.numdays)

In [ ]:
# parallel 
# pool = multiprocessing.Pool(processes=2)
# pool.map(processday_dftraj,daystoprocess)
# serial
for daynum in daystoprocess:
    processday_dftraj(daynum)

day:  2019-08-16 00:00:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
writing to file....
